In [1]:
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
#Indicate list of files names that require data cleaning
filenames = ['L1S12', 'L1S19', 'L2S11', 'L2S18', 'L2S20', 'L3S11', 'L3S13', 'L3S14', 'L3S18']


In [3]:
#convert all files into dataframes for manipulation
dfs = [pd.read_excel(file + '.xlsx') for file in filenames]

In [4]:
#Incorrect column names demonstration
dfs[6]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Date,Time,Record Information,Cycles,Load [kips],Boundary Conditions,Data File Name,MTS File Name,Remarks
1,2012-04-11 00:00:00,01:05:00,0,0,0,baseline,'L3S13_0_0.mat',NaN,baseline
2,2012-04-11 00:00:00,01:07:00,3,0,0,Traction Free,'L3S13_0_3_1.mat',NaN,NaN
3,2012-04-11 00:00:00,01:18:00,2,0,0.001,Clamped,'L3S13_0_2_1.mat',NaN,NaN
4,2012-04-11 00:00:00,01:25:00,1,1,0.5,Loaded,'L3S13_1_1_1.mat',L3_S13_S00,NaN
...,...,...,...,...,...,...,...,...,...
143,2012-06-12 00:00:00,03:12:00,2,300000,-0.001,Clamped,'L3S13_300000_2_1.mat',NaN,NaN
144,2012-06-12 00:00:00,03:16:00,3,300000,0,Traction Free,'L3S13_300000_3_1.mat',NaN,NaN
145,2012-06-13 00:00:00,09:26:00,3,300000,0,Traction Free,'L3S13_300000_3_2.mat',NaN,NaN
146,2012-06-13 00:00:00,09:33:00,2,300000,0.004,Clamped,'L3S13_300000_2_2.mat',NaN,NaN


In [5]:
#Correcting data sets where column names were placed in the second row instead of the first
for num in range(9):
    #checks condition if any of the column names are recorded wrongly
    if any(dfs[num].columns.str.contains('Unnamed')) == True:
        #Corrects column names
        dfs[num].columns = dfs[num].loc[0]
        #Remove the top row that got mistakenly recorded as column names and reset indexing
        dfs[num].drop(index=0, inplace=True)
        dfs[num].reset_index(drop=True, inplace=True)

In [6]:
#Corrected Column names demonstration
dfs[6]

,Date,Time,Record Information,Cycles,Load [kips],Boundary Conditions,Data File Name,MTS File Name,Remarks
0,2012-04-11 00:00:00,01:05:00,0,0,0,baseline,'L3S13_0_0.mat',NaN,baseline
1,2012-04-11 00:00:00,01:07:00,3,0,0,Traction Free,'L3S13_0_3_1.mat',NaN,NaN
2,2012-04-11 00:00:00,01:18:00,2,0,0.001,Clamped,'L3S13_0_2_1.mat',NaN,NaN
3,2012-04-11 00:00:00,01:25:00,1,1,0.5,Loaded,'L3S13_1_1_1.mat',L3_S13_S00,NaN
4,2012-04-11 00:00:00,01:28:00,1,1,1.001,Loaded,'L3S13_1_1_2.mat',NaN,NaN
...,...,...,...,...,...,...,...,...,...
142,2012-06-12 00:00:00,03:12:00,2,300000,-0.001,Clamped,'L3S13_300000_2_1.mat',NaN,NaN
143,2012-06-12 00:00:00,03:16:00,3,300000,0,Traction Free,'L3S13_300000_3_1.mat',NaN,NaN
144,2012-06-13 00:00:00,09:26:00,3,300000,0,Traction Free,'L3S13_300000_3_2.mat',NaN,NaN
145,2012-06-13 00:00:00,09:33:00,2,300000,0.004,Clamped,'L3S13_300000_2_2.mat',NaN,NaN


In [7]:
#Remove redundant columns for each dataframe
for num in range(9):
    #Remove data from the 4th column onwards since it is redundant
    dfs[num].drop(labels=dfs[num].columns[4:], axis=1, inplace=True)
    #Remove data in the 3rd column which is the 'record information' column since it is also redundant 
    dfs[num].drop(labels=dfs[num].columns[2], axis=1, inplace=True)

In [8]:
#Create a 'Component ID' column for each dataframe to identify data associated to each sample
for num in range(9):
    dfs[num]['Component ID'] = filenames[num]

In [9]:
#Create 'State (F/S)' Column to indicate sample state
for num in range(9):
    #indicate all values in the 'State (F/S)'' column of each dataframe to be 'S'
    dfs[num]['State (F/S)'] = 'S'
    #changes the last value in the 'State (F/S)' column to 'F'
    dfs[num].loc[dfs[num].index[-1], 'State (F/S)'] = 'F'

In [10]:
#Standardise Column names across all dataframes
for num in range(9):
    dfs[num].columns = ['Date' , 'Time', 'Cycles', 'Component ID', 'State (F/S)']

In [11]:
#Removing rows that is missing any values in either the 'Date', 'Time' or 'Cycles' column 
for num in range(9):
    dfs[num].dropna(axis='index', how='any', subset=['Date', 'Time', 'Cycles'], inplace=True)

In [12]:
#demonstration of the 'Date' columns having varying data types in different dataframes
dfs[5]['Date']

0     2011-08-01
1     2011-08-01
2     2011-08-01
3     2011-08-01
4     2011-08-01
         ...    
117   2011-10-11
118   2011-10-11
119   2011-10-11
120   2011-10-11
121   2011-10-11
Name: Date, Length: 122, dtype: datetime64[ns]

In [13]:
#demonstration of the 'Date' columns having varying data types in different dataframes
dfs[6]['Date']

0      2012-04-11 00:00:00
1      2012-04-11 00:00:00
2      2012-04-11 00:00:00
3      2012-04-11 00:00:00
4      2012-04-11 00:00:00
              ...         
142    2012-06-12 00:00:00
143    2012-06-12 00:00:00
144    2012-06-13 00:00:00
145    2012-06-13 00:00:00
146    2012-06-13 00:00:00
Name: Date, Length: 147, dtype: object

In [14]:
#standardise date columns of all dataframes to datetime64 type 
for num in range(9):
    if dfs[num]['Date'].dtypes != np.dtype('<M8[ns]'):
        dfs[num]['Date'] = pd.to_datetime(dfs[num]['Date'], format='%Y-%m-%d')

In [15]:
#Convert all date and time columns in the dataframes to str for concatenation
for num in range(9):
    dfs[num][['Date','Time']] = dfs[num][['Date','Time']].astype(str)

In [16]:
#demonstration for time and date values recorded incorrectly
dfs[5].iloc[55:60]

,Date,Time,Cycles,Component ID,State (F/S)
55,2011-08-09,10:17:00,1000,L3S11,S
56,2011-08-09,10:24:00,1000,L3S11,S
57,2011-08-09,10:29:00,1000,L3S11,S
58,2011-08-09,1900-01-11 00:00:00,10000,L3S11,S
59,2011-08-09,11:30:00,10000,L3S11,S


In [17]:
#Create new 'Date_Time' column to store both date and time (errors='coerce arg to handle values that were recorded incorrectly')
for num in range(9):
    dfs[num]['Date_Time'] = pd.to_datetime(dfs[num]['Date'] + ' ' + dfs[num]['Time'],errors='coerce',format='%Y-%m-%d %H:%M:%S')

In [18]:
#demonstration for time and date values recorded incorrectly after using .to_datetime method
dfs[5].iloc[55:60]

,Date,Time,Cycles,Component ID,State (F/S),Date_Time
55,2011-08-09,10:17:00,1000,L3S11,S,2011-08-09 10:17:00
56,2011-08-09,10:24:00,1000,L3S11,S,2011-08-09 10:24:00
57,2011-08-09,10:29:00,1000,L3S11,S,2011-08-09 10:29:00
58,2011-08-09,1900-01-11 00:00:00,10000,L3S11,S,NaT
59,2011-08-09,11:30:00,10000,L3S11,S,2011-08-09 11:30:00


In [19]:
#remove incorrect values in 'Date_Time' column by filtering incorrect values out of the dataframes
for num in range(9):
    filt = dfs[num]['Date_Time'].notnull() == True
    dfs[num] = dfs[num][filt]

In [20]:
#demonstration of incorrect values in 'Date_Time' column being removed 
dfs[5].iloc[55:60]

,Date,Time,Cycles,Component ID,State (F/S),Date_Time
55,2011-08-09,10:17:00,1000,L3S11,S,2011-08-09 10:17:00
56,2011-08-09,10:24:00,1000,L3S11,S,2011-08-09 10:24:00
57,2011-08-09,10:29:00,1000,L3S11,S,2011-08-09 10:29:00
59,2011-08-09,11:30:00,10000,L3S11,S,2011-08-09 11:30:00
60,2011-08-09,11:34:00,10000,L3S11,S,2011-08-09 11:34:00


In [21]:
#removing 'Date' and 'Time' columns since they are no longer needed
for num in range(9):
    dfs[num] = dfs[num].drop(columns=['Date', 'Time'])

In [22]:
#creating new 'Time_Passed (min)'' column to store the time that has passed since start of experiment in minutes
for num in range(9):
    #obtain time difference between starting time and measurement time
    dfs[num]['Time_Passed (min)'] = dfs[num].loc[:, 'Date_Time'] - dfs[num].loc[0,'Date_Time']
    #covert time difference into minutes and remove decimal points
    dfs[num]['Time_Passed (min)'] = dfs[num]['Time_Passed (min)'].apply(lambda x: x.total_seconds()) / 60
    dfs[num]['Time_Passed (min)'] = dfs[num]['Time_Passed (min)'].apply(lambda x: math.trunc(x))

In [23]:
#Remove 'Date_Time' column since it is no longer needed anymore
for num in range(9):
    dfs[num].drop(columns='Date_Time', inplace=True)

In [24]:
#Demonstration of a cleaned dataframe
dfs[6]

,Cycles,Component ID,State (F/S),Time_Passed (min)
0,0,L3S13,S,0
1,0,L3S13,S,2
2,0,L3S13,S,13
3,1,L3S13,S,20
4,1,L3S13,S,23
...,...,...,...,...
142,300000,L3S13,S,89407
143,300000,L3S13,S,89411
144,300000,L3S13,S,91221
145,300000,L3S13,S,91228


In [25]:
#Merge all the dataframes with individual sample data into a single dataframe
CFRP_Cleaned_Data = pd.DataFrame()
for num in range(9):
    CFRP_Cleaned_Data = CFRP_Cleaned_Data.append(dfs[num], ignore_index=True)

In [26]:
#Check info of the newly merge data set
CFRP_Cleaned_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cycles             1071 non-null   object
 1   Component ID       1071 non-null   object
 2   State (F/S)        1071 non-null   object
 3   Time_Passed (min)  1071 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 33.6+ KB


In [27]:
#Demonstration of how the merged dataframe looks like
CFRP_Cleaned_Data

,Cycles,Component ID,State (F/S),Time_Passed (min)
0,0,L1S12,S,0
1,0,L1S12,S,19
2,1,L1S12,S,32
3,1,L1S12,S,34
4,1,L1S12,S,37
...,...,...,...,...
1066,750000,L3S18,S,23051
1067,800000,L3S18,S,23241
1068,850000,L3S18,S,22718
1069,850000,L3S18,S,22718


In [28]:
#Export the cleaned data into a new excel file
CFRP_Cleaned_Data.to_excel('CFRP_Cleaned_Data.xlsx')